In [ ]:
!pip install py-entitymatching

     |████████████████████████████████| 2.0MB 4.0MB/s 
     |████████████████████████████████| 788kB 33.8MB/s 
     |████████████████████████████████| 1.1MB 36.4MB/s 
     |████████████████████████████████| 368kB 42.4MB/s 
     |████████████████████████████████| 665kB 38.4MB/s 
  Created wheel for py-entitymatching: filename=py_entitymatching-0.4.0-cp37-cp37m-linux_x86_64.whl size=2634111 sha256=ee6835a44fbf9b75a8611a5f54558adaa1ec7abf02f70f7335934fde2a6072b3
  Stored in directory: /root/.cache/pip/wheels/88/65/50/a8bc0f26e8a82c931658153c4f37cbf5debc22cb0f61dbcd05
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp37-cp37m-linux_x86_64.whl size=3723831 sha256=2ed200d7cd891692ca39606df413f3ffd712b7d4c8cb51ad86652f30ff9c8686
  Stored in directory: /root/.cache/pip/wheels/58/30/8e/6fa463100ff6e8595b131067ec43a118c607f621ac297e7dad
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp37-cp37m-linux_x86_64.whl size=2055267 sha256=b21bf72258b43594644

In [ ]:
# https://hpi.de/naumann/projects/repeatability/datasets/restaurants-dataset.html
# http://www.iesl.cs.umass.edu/datasets.html - CORA dataset
# Importing the libraries
import sys
import pandas as pd
import os
from google.colab import drive
from math import *
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
path_variable = "/content/gdrive/My Drive/BTP/Code/Dataset/Restaurant/"

In [ ]:
import py_entitymatching as em
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
pandas version: 1.1.5
magellan version: 0.4.0


In [ ]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = path_variable+"fodors.csv"
path_B = path_variable+"zagats.csv"
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')
# Display number of tuples in the datasets
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


Number of tuples in A: 533
Number of tuples in B: 331
Number of tuples in A X B (i.e the cartesian product): 176423


In [ ]:
# Displaying first two entries from the first restaurant dataset
A.head(2)

,id,name,addr,city,phone,type,class,group
0,534,'arnie morton\'s of chicago','435 s. la cienega blv.','los angeles',310/246-1501,american,0,American
1,535,'art\'s delicatessen','12224 ventura blvd.','studio city',818/762-1221,american,1,American


In [ ]:
# Displaying first two entries from the second restaurant dataset
B.head(2)

,id,name,addr,city,phone,type,class,group
0,1,'apple pan the','10801 w. pico blvd.','west la',310-475-3585,american,534,American
1,2,'asahi ramen','2027 sawtelle blvd.','west la',310-479-2231,'noodle shops',535,Asian


**Blocking**

In [ ]:
# Overlap based blocker, which blocks on the basis of overlap of some qgram between two strings
ob = em.OverlapBlocker()
C = ob.block_tables(A, B, "addr", "addr", overlap_size=2, 
                    l_output_attrs=["id", "name", "addr","city","phone","type", "class","group"], 
                    r_output_attrs=["id", "name", "addr","city","phone","type", "class","group"]
                    )


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
# Printing length of candidate set
print(len(C))
print(C.head(2))

4951
   _id  ltable_id  rtable_id  ... rtable_type rtable_class rtable_group
0    0        555          1  ...    american          534     American
1    1        663          1  ...    american          534     American

[2 rows x 17 columns]


**Feature Generation**

In [ ]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [ ]:
# Manually creating the phone feature

def jaccard_sim(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def phone_phone_lev_sim(ltuple, rtuple):
    p1 = ltuple.phone
    p2 = rtuple.phone
    return em.lev_sim(p1,p2)

def phone_phone_cos(ltuple, rtuple):
    p1 = ltuple.phone
    p2 = rtuple.phone
    return em.cosine(list(p1),list(p2))

def phone_phone_jac(ltuple, rtuple):
    p1 = ltuple.phone
    p2 = rtuple.phone
    return jaccard_sim(p1,p2)

def phone_phone_mel(ltuple,rtuple):
    p1 = ltuple.phone
    p2 = rtuple.phone
    if em.monge_elkan(list(p1),list(p2))=="nan":
        return 0
    else:
        return em.monge_elkan(list(p1),list(p2))

em.add_blackbox_feature(feature_table, 'phone_phone_lev_sim', phone_phone_lev_sim)
em.add_blackbox_feature(feature_table, 'phone_phone_cos', phone_phone_cos)
em.add_blackbox_feature(feature_table, 'phone_phone_jac', phone_phone_jac)
em.add_blackbox_feature(feature_table, 'phone_phone_mel', phone_phone_mel)

True

In [ ]:
feature_table['feature_name']

0                           id_id_exm
1                           id_id_anm
2                      id_id_lev_dist
3                       id_id_lev_sim
4           name_name_jac_qgm_3_qgm_3
5       name_name_cos_dlm_dc0_dlm_dc0
6       name_name_jac_dlm_dc0_dlm_dc0
7                       name_name_mel
8                  name_name_lev_dist
9                   name_name_lev_sim
10                      name_name_nmw
11                       name_name_sw
12          addr_addr_jac_qgm_3_qgm_3
13      addr_addr_cos_dlm_dc0_dlm_dc0
14      addr_addr_jac_dlm_dc0_dlm_dc0
15                      addr_addr_mel
16                 addr_addr_lev_dist
17                  addr_addr_lev_sim
18                      addr_addr_nmw
19                       addr_addr_sw
20          city_city_jac_qgm_3_qgm_3
21      city_city_cos_dlm_dc0_dlm_dc0
22      city_city_jac_dlm_dc0_dlm_dc0
23                      city_city_mel
24                 city_city_lev_dist
25                  city_city_lev_sim
26          

In [ ]:
# Only take features corresponding to the four sim scores : cosine, Jaccard, lev_sim, monge_elkan
feature_table = feature_table.drop([0,1,2,3,6,8,10,11,14,16,18,19,22,24,26,27,30,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47],axis=0)
feature_table['feature_name']

In [ ]:
H = em.extract_feature_vecs(C, feature_table=feature_table)
H

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:24


,_id,ltable_id,rtable_id,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_sim,addr_addr_jac_qgm_3_qgm_3,addr_addr_cos_dlm_dc0_dlm_dc0,addr_addr_mel,addr_addr_lev_sim,city_city_jac_qgm_3_qgm_3,city_city_cos_dlm_dc0_dlm_dc0,city_city_mel,city_city_lev_sim,type_type_jac_qgm_3_qgm_3,type_type_cos_dlm_dc0_dlm_dc0,type_type_mel,type_type_lev_sim,phone_phone_lev_sim,phone_phone_cos,phone_phone_jac,phone_phone_mel
0,0,555,1,0.000000,0.0,0.622685,0.312500,0.448276,0.577350,0.778992,0.666667,0.125000,0.0,0.678571,0.357143,0.117647,0.0,0.690476,0.375000,0.333333,0.707107,0.545455,0.750000
1,1,663,1,0.088235,0.0,0.627573,0.294118,0.333333,0.500000,0.822015,0.565217,0.083333,0.0,0.688462,0.307692,0.000000,0.0,0.301282,0.076923,0.166667,0.707107,0.545455,0.666667
2,2,677,1,0.051282,0.0,0.589286,0.238095,0.642857,0.750000,0.955556,0.857143,0.083333,0.0,0.688462,0.307692,0.000000,0.0,0.301282,0.076923,0.333333,0.707107,0.545455,0.583333
3,3,549,4,0.111111,0.0,0.583750,0.250000,0.564103,0.670820,0.799818,0.687500,0.103448,0.0,0.669615,0.266667,0.000000,0.0,0.583333,0.222222,0.333333,0.668153,0.500000,0.750000
4,4,665,4,0.000000,0.0,0.484722,0.125000,0.512195,0.670820,0.764642,0.687500,0.103448,0.0,0.669615,0.266667,0.057143,0.0,0.694242,0.277778,0.583333,0.801784,0.666667,0.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4946,4946,764,331,0.045455,0.0,0.548238,0.241379,0.333333,0.577350,0.850826,0.578947,0.000000,0.0,0.414286,0.000000,0.000000,0.0,0.472222,0.125000,0.166667,0.645497,0.454545,0.416667
4947,4947,920,331,0.000000,0.0,0.532471,0.137931,0.653846,0.666667,0.959298,0.850000,1.000000,1.0,1.000000,1.000000,0.000000,0.0,0.640491,0.312500,0.500000,0.577350,0.400000,0.666667
4948,4948,930,331,0.035088,0.0,0.592175,0.172414,0.444444,0.471405,0.807470,0.516129,1.000000,1.0,1.000000,1.000000,0.000000,0.0,0.520833,0.062500,0.416667,0.680414,0.500000,0.583333
4949,4949,945,331,0.068182,0.0,0.658842,0.344828,0.348837,0.235702,0.789774,0.457143,1.000000,1.0,1.000000,1.000000,0.272727,0.0,0.833333,0.500000,0.333333,0.433013,0.272727,0.416667


In [ ]:
import pickle

dbfile = open(path_variable+'files/feature_matrix_restaurant', 'wb') 
pickle.dump(H, dbfile) 
dbfile.close()

candidate_set = open(path_variable+'files/C_restaurant', 'wb') 
pickle.dump(C, candidate_set)
candidate_set.close()